In [3]:
import json
import pandas as pd
from pandas import DataFrame
from pandas.io.json import json_normalize #package for flattening json in pandas df

In [8]:
# COPY META COLUMNS TO PERIOD RESULTS DATAFRAME
# a: 1-row DataFrame containing Period and meta columns
# b: results DataFrame
def copy_meta_to_period(a, b):
    headers = list(a)
    for i in range(1,len(headers)):
        b[str(headers[i])] = a.iloc[0][i]
    return b

def handle_lists(x, i):
    a = DataFrame(x.iloc[i].to_dict())
    b = json_normalize(data=a['Period'], record_path=['Rep'], meta=['type', 'value'])
    return a, b

def handle_objects(x, i):
    a = DataFrame(x.iloc[i])
    b = json_normalize(data=a.T['Period'], record_path=['Rep'], meta=['type', 'value'])
    return a, b

def weather_json2csv(file_path):
    with open(file_path) as f:
        d = pd.io.json.loads(f)

    data_by_location = json_normalize(data=d['SiteRep']['DV'], 
                                      record_path=['Location'],
                                      meta=['dataDate', 'type'])
    
    result = DataFrame()
    # Loop over all rows of location
    for i in range (0, len(data_by_location)):

        if (isinstance(data_by_location.iloc[i]['Period'], list)): # Handle lists
            [a, b] = handle_lists(data_by_location, i)
        else:                                              # Handle objects
            [a, b] = handle_objects(data_by_location, i)

        if (len(result) == 0):
            result = copy_meta_to_period(a, b)
        else:
            result = result.append(copy_meta_to_period(a, b), ignore_index=True)

    return result.to_csv('result.csv')    

In [9]:
weather_json2csv('./test_data/weather_obs-1.json')

TypeError: Expected String or Unicode